In [1]:
drive = False  # False for Local
if drive:
    !pip install pretty_midi

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pretty_midi
from mido import Message, MidiFile, MidiTrack
import pandas as pd
import json
import torch as torch
from tqdm import tqdm

In [3]:
if drive:
    from google.colab import drive
    drive.mount('/content/drive')
    my_drive_path = '/content/drive/MyDrive/2022_Special_Studies_Hablutzel/ModelCopies/'
else: # local
    my_drive_path = "./"

In [4]:
my_data = pd.read_csv(my_drive_path + 'maestro-v3.0.0/maestro-v3.0.0.csv')

In [5]:
my_data.iloc[45]

canonical_composer                                       Claude Debussy
canonical_title                          "Voiles" from Preludes, Book I
split                                                             train
year                                                               2008
midi_filename         2008/MIDI-Unprocessed_07_R3_2008_01-05_ORIG_MI...
audio_filename        2008/MIDI-Unprocessed_07_R3_2008_01-05_ORIG_MI...
duration                                                     212.295461
Name: 45, dtype: object

In [6]:
# how many beats/note onsets to use
target_size = 800
uniform_piece_size = False  # false for variable-length pieces
repeat_if_too_short = False
use_only_first_piece = True
round_up = False
round_down = True
round_interval = 30
path_beginning = my_drive_path + "maestro-v3.0.0/"
train = []
test = []
validation = []
data_sizes = []

#how many steps you get
for i in tqdm(range(0, my_data.shape[0])):  # 1276 pieces
  #loads the piece into pretty_midi form
  path_part = my_data.iloc[i]["midi_filename"]
  split = my_data.iloc[i]["split"]
  path = path_beginning + path_part
  midi_data = pretty_midi.PrettyMIDI(path)

  #estimates the tempo of the piece
  tempo = midi_data.estimate_tempo()
  print(" estimated tempo:", tempo, end="")
  #gets piano roll sampled at estimated tempo
  #(diff tempo for each piece)
  roll = midi_data.get_piano_roll(tempo/60).T
  print("\tbeats:", roll.shape[0], end=" -> ")

  # removes silence at beginning of piece
  over = False
  r = np.zeros(128)
  while over == False:
    m = roll[0]
    if np.array_equal(m,r):
      roll = roll[1:]
    else:
      over = True
    
  # removes silence at end of piece
  over = False
  r = np.zeros(128)
  while over == False:
    m = roll[-1]
    if np.array_equal(m,r):
      roll = roll[:-1]
    else:
      over = True

  # adds silence at the end of piece, so length is multiple of round_interval
  # print(roll.shape)
  if round_up:
    beats_to_add = (roll.shape[0] * -1) % round_interval  # example, -4 mod 10 gives 6 beats to add
    pad = np.zeros((beats_to_add, 128))
    roll = np.vstack([roll, pad])
  # remove last few beats so length is multiple of round_interval
  elif round_down:
    beats_to_remove = roll.shape[0] % round_interval
    if beats_to_remove > 0:
        roll = roll[:(-1) * beats_to_remove] # cut off last [beats_to_remove] beats

  # recalculate num beats after removing silence etc.
  beats = roll.shape[0]
  print(beats)  # ends print line from above (pre-processing beats)

  #######
  # save data on preprocessed piece sizes
  #######
  this_piece = my_data.iloc[i].copy()
  this_piece["tempo"] = tempo
  this_piece["beats"] = beats
  data_sizes.append(this_piece)

  #converts roll to binary matrix (only 0s and 1s)
  roll = np.where(roll > 0, 1, 0)

  sev_roll = [] # store pieces

  if not uniform_piece_size:
    # save whole piece
    sev_roll.append(np.array([roll,tempo,beats], dtype="object"))
  else:
    # separate roll into pieces of target_size each
    if roll.shape[0]<target_size:
      if repeat_if_too_short:
        #if the piece is shorter than the target size, repeat the piece until it reaches the target size.
        while roll.shape[0]<target_size:
          roll = np.vstack((roll,roll))
        roll = roll[:target_size]
        sev_roll.append(np.array([roll,tempo,beats], dtype="object"))
      else:
        continue  # skip pieces less than target size
    else:
      if use_only_first_piece:
        # just keep the first piece
        num_arrays = 1
        leftover = 0
      else:
        #otherwise, divide up the piece into target-sized smaller pieces. If the remainder are longer than half the target size, make that a new piece as well.
        num_arrays = roll.shape[0]//target_size  # target_size pieces
        leftover = roll.shape[0]%target_size  # leftover beyond target_size pieces

      # get target_size pieces
      for l in range(0, num_arrays):
        start = target_size*l
        end = target_size*(l+1)
        sev_roll.append(np.array([roll[start:end],tempo,beats], dtype="object"))

      # if leftover larger than half the target size, repeat and use as well
      if leftover >= target_size//2:
        if repeat_if_too_short:
          leftover_roll = roll[end:]
          while leftover_roll.shape[0]<target_size:
            leftover_roll = np.vstack((leftover_roll,leftover_roll))
          leftover_roll = leftover_roll[:target_size]
          sev_roll.append(np.array([leftover_roll,tempo,beats], dtype="object"))

  # for each roll from this piece
  for roll in sev_roll:
    #add this into the correct file.
    roll = np.array([torch.tensor(roll[0]), roll[1], roll[2]], dtype="object") # convert first element to tensor
    if split=="test":
      test.append(roll)
    elif split=="train":
      train.append(roll)
    else:
      validation.append(roll)

#### for saving data sizes
data_sizes_df = pd.DataFrame.from_records(data_sizes)
data_sizes_df.head()

  0%|          | 1/1276 [00:00<07:23,  2.87it/s]

 estimated tempo: 187.51378314519036	beats: 2200 -> 2190


  0%|          | 2/1276 [00:00<07:44,  2.74it/s]

 estimated tempo: 166.18828503917456	beats: 2118 -> 2100


  0%|          | 3/1276 [00:01<07:07,  2.98it/s]

...

100%|█████████▉| 1275/1276 [06:22<00:00,  3.01it/s]

 estimated tempo: 196.33873544853319	beats: 3498 -> 3480


100%|██████████| 1276/1276 [06:22<00:00,  3.34it/s]

 estimated tempo: 212.26533166457037	beats: 2231 -> 2220


,canonical_composer,canonical_title,split,year,midi_filename,audio_filename,duration,tempo,beats
0,Alban Berg,Sonata Op. 1,train,2018,2018/MIDI-Unprocessed_Chamber3_MID--AUDIO_10_R...,2018/MIDI-Unprocessed_Chamber3_MID--AUDIO_10_R...,698.661160,187.513783,2190
1,Alban Berg,Sonata Op. 1,train,2008,2008/MIDI-Unprocessed_03_R2_2008_01-03_ORIG_MI...,2008/MIDI-Unprocessed_03_R2_2008_01-03_ORIG_MI...,759.518471,166.188285,2100
2,Alban Berg,Sonata Op. 1,train,2017,2017/MIDI-Unprocessed_066_PIANO066_MID--AUDIO-...,2017/MIDI-Unprocessed_066_PIANO066_MID--AUDIO-...,464.649433,195.593838,1500
3,Alexander Scriabin,"24 Preludes Op. 11, No. 13-24",train,2004,2004/MIDI-Unprocessed_XP_21_R1_2004_01_ORIG_MI...,2004/MIDI-Unprocessed_XP_21_R1_2004_01_ORIG_MI...,872.640588,202.212360,2910
4,Alexander Scriabin,"3 Etudes, Op. 65",validation,2006,2006/MIDI-Unprocessed_17_R1_2006_01-06_ORIG_MI...,2006/MIDI-Unprocessed_17_R1_2006_01-06_ORIG_MI...,397.857508,204.545455,1350


In [7]:
#the data does not save until you run this.
torch.save(validation, my_drive_path + "usable_data/validation_tempo_round_down_30.csv" )
torch.save(test, my_drive_path + "usable_data/test_tempo_round_down_30.csv" )
torch.save(train, my_drive_path + "usable_data/train_tempo_round_down_30.csv" )

In [8]:
data_sizes_df.to_csv(my_drive_path + "usable_data/data_sizes.csv")

In [9]:
# see how many pieces are in each set
print("validation:\t", len(validation))
print("test:\t", len(test))
print("train:\t", len(train))

validation:	 137
test:	 177
train:	 962


In [10]:
#### test code: see what percent of pieces are at least ## beats
new_size = 2000
new_df = data_sizes_df[data_sizes_df['beats'] >= new_size]
print(new_df.shape)
new_df.shape[0] / data_sizes_df.shape[0]
# two pieces are size 595 - including them we have 89.7% of pieces
# 600 keeps 89.49% of our data - 1142 pieces
# 700 keeps 86%
# 800 keeps 80% - 1022 pieces
# 900 keeps 75%
# 1000 keeps 69% - 878 pieces
# 1100 keeps 64%


# how many would have at least two full chunks?
# 1200 : 59%
# 1600 : 46%
# 2000 : 34%

(434, 9)


0.3401253918495298